### Model analysis

As coded by ChatGPT

In [1]:
import cobra
import pandas as pd
import os
import json

# Load the uptake rates from the provided CSV file
uptake_rates_file = "datasets/Estimated_Uptake_Rates.csv"
uptake_rates = pd.read_csv(uptake_rates_file, index_col="label")

In [2]:
# Add repeated metabolites
uptake_rates = uptake_rates.groupby("Metabolite ID").sum()

In [3]:
# List of metabolic model files (Assumed JSON format)
model_files = [
    "Marmoricola_OAE513.json",
    "Mycobacterium_OAE908.json",
    "Mucilaginibacter_OAE612.json",
    "Arthrobacter_OAP107.json",
    "Rhodococcus_OAS809.json",
    "Bacillus_OAE603.json",
    "Chitinophagaceae_OAS944.json",
    "Methylobacterium_OAE515.json",
    "Bradyrhizobium_OAE829.json",
    "Chitinophaga_OAE865.json",
    "Paenibacillus_OAE614.json",
    "Lysobacter_OAE881.json",
    "Bosea_OAE506.json",
    "Rhizobium_OAE497.json",
    "Burkholderia_OAS925.json",
    "Variovorax_OAS795.json",
    "Brevibacillus_OAP136.json"
]

# Load models, apply uptake rates, and calculate individual flux distributions
individual_flux_results = {}
models = []
for model_file in model_files:
    model_file = "17mmodels/" + model_file
    if os.path.exists(model_file):
        model = cobra.io.load_json_model(model_file)
        
        # Apply uptake rates to exchange reactions
        for met_id, row in uptake_rates.iterrows():
            if not isinstance(met_id,str):
                continue
            rxn_id = "EX_" + met_id
            if rxn_id in model.reactions:
                model.reactions.get_by_id(rxn_id).lower_bound = -row["Uptake Rate"]
        
        # Perform FBA for the individual model
        solution = model.optimize()
        
        # Store results
        individual_flux_results[model.id] = {
            "Reaction": [rxn.id for rxn in model.reactions],
            "Flux": [solution.fluxes[rxn.id] for rxn in model.reactions]
        }
        
        models.append(model)
    else:
        print(f"Warning: Model file {model_file} not found.")

In [4]:
# Save individual flux results
for model_id, flux_data in individual_flux_results.items():
    df = pd.DataFrame(flux_data)
    df.to_csv(f"modeling_datasets/fluxes-single/{model_id}_flux_results.csv", index=False)